<a href="https://colab.research.google.com/github/muhammadnouman911/Face-Detection-Classification-Regression/blob/main/Face_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mtcnn tensorflow keras-facenet scikit-learn opencv-python

from mtcnn import MTCNN
from keras_facenet import FaceNet
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import os

# Initialize models
detector = MTCNN()
embedder = FaceNet()

# Example training data (replace with your dataset)
# Each entry: ("path_to_face_image", "label")
train_data = [
    ("/content/drive/MyDrive/dataset/faces/-wuxbgMRIWs_30.03_36.745044/frame_0000.jpg", "Alice"),
    ("/content/drive/MyDrive/dataset/faces/1uizi6R-PMU_116.92_120.0/frame_0001.jpg", "Bob")
]

# Extract embeddings for training data
X_train = []
y_train = []
for img_path, label in train_data:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading training image: {img_path}")
        continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img_rgb)
    if faces:
        x, y, w, h = faces[0]['box']
        face = img_rgb[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))
        embedding = embedder.embeddings([face])[0]
        X_train.append(embedding)
        y_train.append(label)

# Train classifier
if len(X_train) > 0:
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)

    # Test on new image
    test_img_path = "test.jpg"  # Assuming test.jpg is in the same directory or adjust path
    test_img = cv2.imread(test_img_path)

    if test_img is None:
        print(f"Error: Could not load test image from {test_img_path}. Make sure the file exists and the path is correct.")
    else:
        test_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(test_rgb)
        for face in faces:
            x, y, w, h = face['box']
            face_crop = test_rgb[y:y+h, x:x+w]
            face_crop = cv2.resize(face_crop, (160, 160))
            embedding = embedder.embeddings([face_crop])[0]
            pred_label = knn.predict([embedding])[0]
            cv2.putText(test_img, pred_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.rectangle(test_img, (x, y), (x+w, y+h), (0,255,0), 2)

        cv2.imshow("Face Recognition", test_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
else:
    print("No training data loaded. Cannot train the classifier.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Error: Could not load test image from test.jpg. Make sure the file exists and the path is correct.
